In [1]:
import requests
import pandas as pd

stationen: https://planner.dbcargo.com/stations.json

- erstmal alle Stationen darüber bekommen
    - nach german filtern
- erweiterte ids herausfinden
    - stationen ohne routingcode haben meistens keine verbindungen (have yet to find some)
    - stationen ohne routingcode kann man rausschmeißen
- übriggebliebenen durchsuchen, quadratische laufzeit maybe
- es gibt stationen die haben routingcode xxx-00, 
    - andere stationen haben xxx-yy, diese sind maybe unterBahnhöfe von den -00 dingern, vllt kann man die auch skippen


In [2]:
def get_stations():
    headers = {}
    search_url = f"https://planner.dbcargo.com/stations.json"
    params = {}
    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200:
        d = response.json()
    return d


In [3]:
def get_departing(id, country = 80):
    base = "https://planner.dbcargo.com/nwd/commercialLocations"
    params = {
        "countryNumber": country,
        "locationNumber" : id,
        "locationType" : "Dispatch",
        "mappingType" : "Partner"
    }
    response = requests.get(url = base, params=params)
    d = None
    if response.status_code == 200:
        return response
    return None

In [4]:
def dataframe_response(data):
    def unpack_single_element_lists(d):
        if isinstance(d, dict):
            return {k: unpack_single_element_lists(v) for k, v in d.items()}
        elif isinstance(d, list) and len(d) > 0:
            return unpack_single_element_lists(d[0])  # Nimmt das einzige Element
        return d  # Andernfalls unverändert zurückgeben

    data2 = unpack_single_element_lists(data)

    # JSON normalisieren
    normalized_data = pd.json_normalize(data2)
    return normalized_data




In [5]:


payload = {
    "arrivalLocationId" : 22922245,
    "departureLocationId": 22920962,
    "dispatchLocationId" : 22920961,
    "earliestTimeOfDay" : 0,
    "latestTimeOfDay" : 86399,
    "mappingType": "Partner",
    "receivingLocationId" : 22929561,
    "routingCode" : "630-00",
    "scenarioHash": "2d2996d25ff0d332744c3d1310942e8dc6256396d3432aff986b6664359047ed",
    "searchMode": "Departure",
    "weekdays": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
}
base = "https://planner.dbcargo.com/nwd/timetableQuery"

# response = requests.post(url=base,json= payload)
# res = response.json()

dispatch = locations.id 
departure = locations.production....id 
receiving = locations.id
arrival = locations.production....id
routing Code = code vom receiving end

In [6]:
def get_connection_synchronus(send1 , send2, rec1, rec2, code):
    base = "https://planner.dbcargo.com/nwd/timetableQuery"
    payload = {
        "dispatchLocationId" : str(send1),
        "departureLocationId": str(send2),
        "receivingLocationId" : str(rec1),
        "arrivalLocationId" : str(rec2),
        "routingCode" : str(code),
        "earliestTimeOfDay" : 0,
        "latestTimeOfDay" : 86399,
        "mappingType": "Partner",
        "scenarioHash": "2d2996d25ff0d332744c3d1310942e8dc6256396d3432aff986b6664359047ed",
        "searchMode": "Departure",
        "weekdays": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    }

    response = requests.post(url=base,json= payload)
    return response

In [7]:
stations = get_stations()
df = pd.DataFrame(stations)


In [8]:
german = df[df["country"] == 80]
german = german.drop("country",axis = 1).reset_index().drop("index",axis = 1)

In [9]:
german

,id,name
0,010033,Hamburg-Who Ct Burchardkai Ctb
1,010066,Kudensee
2,010116,Maschen MVZ
3,010124,Maschen Rbf Werk Y-Schad
4,010660,Hamburg-Billwerder Ubf
...,...,...
2209,999953,MALUS
2210,999961,KULANZ
2211,999979,LOGISTIK
2212,999987,CZ BLN SERVICE ABEGL


In [10]:
import time
#  this will take long, like 70 minutes for all german stations
def get_consumer_stations(stations):
    results = []
    for k,row in stations.iterrows():
        data = get_departing(row["id"])
        print(row["id"])
        print(k)

        if data is None:
            break

        print(data.json())
        counter = 0
        # time.sleep(1.5 ** counter)
        while "scenarioHash" not in data.json().keys():
            time.sleep(1.5 ** counter)
            data = get_departing(row["id"])
            counter += 1

        print(data.json())

        df = dataframe_response(data.json())
        print(f"length : {len(df.columns)}")
        results.append(df)
    return results

results = []
# results = get_consumer_stations(german)


In [11]:
# testdf = pd.concat(results)


In [12]:
# testdf.to_pickle("company_locations.pkl")

In [13]:
new_df = pd.read_pickle("company_locations.pkl")

In [14]:
new_df = new_df.reset_index().drop("index",axis = 1)
new_df

,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,locations.productionLocationAndRoutingCodes.productionLocation.locationNumber,locations.productionLocationAndRoutingCodes.productionLocation.name,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,010819,HAMBURG-WALTERSHOF,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
1,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010066,None,AKUD,22920361.0,80,010066,KUDENSEE,...,22920413.0,80,010066,KUDENSEE,53.922657,9.219641,KUDENSEE,1.0,NaN,NaN
2,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,011759,MASCHEN RBF,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,011759,MASCHEN RBF,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,010660,HAMBURG-BILLWERDER,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2210,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2211,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2212,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [15]:
# combined = pd.concat([german,new_df],axis=1)
# combined.to_pickle("complete.pkl")

In [16]:
# combined.to_json("complete.json",orient="records")
# combined.to_csv("complete.csv",index = False)
# combined

combined = pd.read_pickle("complete.pkl")

In [17]:
new = combined.dropna(subset= ["locations.productionLocationAndRoutingCodes.routingCodes"])
new

,id,name,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,010033,Hamburg-Who Ct Burchardkai Ctb,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
1,010066,Kudensee,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010066,None,AKUD,22920361.0,80,...,22920413.0,80,010066,KUDENSEE,53.922657,9.219641,KUDENSEE,1.0,NaN,NaN
2,010116,Maschen MVZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,010124,Maschen Rbf Werk Y-Schad,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,010660,Hamburg-Billwerder Ubf,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,893651,Dingolfing PA Scholz,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00893651,None,MADS,22915095.0,80,...,22915096.0,80,893651,DINGOLFING PA SCHOLZ,48.642839,12.491984,DINGOLFING PA SCHOLZ,1.0,NaN,NaN
2201,894725,Bülstringen Florastrasse,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00894725,None,LFLO,22915092.0,80,...,22915093.0,80,894725,BUELSTRINGEN FLORAST,52.311128,11.333092,BUELSTRINGEN FLORAST,1.0,NaN,NaN
2202,897926,Dabel,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00897926,None,WDL,22915088.0,80,...,22915089.0,80,897926,DABEL,53.671404,11.893348,DABEL,2.0,NaN,NaN
2203,899385,Krefeld Industriebahn,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00899385,None,KKRK,22919136.0,80,...,22930374.0,80,899385,KREFELD INDUSTRIEBHN,51.342592,6.591727,KREFELD INDUSTRIEBHN,1.0,NaN,NaN


In [18]:
companys = new.drop_duplicates(subset=["locations.productionLocationAndRoutingCodes.routingCodes"], keep="last")

In [19]:
companys = companys.drop_duplicates(subset= ["locations.productionLocationAndRoutingCodes.productionLocation.latitude"],keep="last")

In [20]:
companys["locations.id"] = companys["locations.id"].astype(int)
companys["locations.productionLocationAndRoutingCodes.productionLocation.id"] = companys["locations.productionLocationAndRoutingCodes.productionLocation.id"].astype(int)
companys

,id,name,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
1,010066,Kudensee,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010066,None,AKUD,22920361,80,...,22920413,80,010066,KUDENSEE,53.922657,9.219641,KUDENSEE,1.0,NaN,NaN
9,011205,Hamburg-Barmbek,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011205,None,ABAG,22920426,80,...,22920427,80,011205,HAMBURG-BARMBEK,53.597759,10.036916,HAMBURG-BARMBEK,1.0,NaN,NaN
12,011460,Dannenberg Ost,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011460,None,ADB,22917069,80,...,22929266,80,011460,DANNENBERG OST,53.107161,11.120125,DANNENBERG OST,1.0,NaN,NaN
13,011742,Lüneburg,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011742,None,ALBG,22920421,80,...,22929263,80,011742,LUENEBURG,53.250934,10.420770,LUENEBURG,2.0,NaN,NaN
16,011833,Meckelfeld,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011833,None,AMD,22920462,80,...,22920463,80,011833,MECKELFELD,53.421757,10.029219,MECKELFELD,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,893651,Dingolfing PA Scholz,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00893651,None,MADS,22915095,80,...,22915096,80,893651,DINGOLFING PA SCHOLZ,48.642839,12.491984,DINGOLFING PA SCHOLZ,1.0,NaN,NaN
2201,894725,Bülstringen Florastrasse,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00894725,None,LFLO,22915092,80,...,22915093,80,894725,BUELSTRINGEN FLORAST,52.311128,11.333092,BUELSTRINGEN FLORAST,1.0,NaN,NaN
2202,897926,Dabel,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00897926,None,WDL,22915088,80,...,22915089,80,897926,DABEL,53.671404,11.893348,DABEL,2.0,NaN,NaN
2203,899385,Krefeld Industriebahn,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00899385,None,KKRK,22919136,80,...,22930374,80,899385,KREFELD INDUSTRIEBHN,51.342592,6.591727,KREFELD INDUSTRIEBHN,1.0,NaN,NaN


In [21]:
Leipig = companys[companys["name"].str.contains("Wahren", na=False)].iloc[0]

Berlin = companys[companys["name"].str.contains("München Nord", na=False)].iloc[0]


Berlin

id                                                                                                                                    202028
name                                                                                                                  München Nord Werkstatt
scenarioHash                                                                               2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...
locations.companyLocationNumberOwner                                                                                                86353173
locations.type                                                                                                                          None
locations.tafTsiCode                                                                                                                00202028
locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber                                                 None
locations.pro

In [22]:
(Berlin["locations.productionLocationAndRoutingCodes.productionLocation.id"])

22922245

In [23]:
companys.iloc[0]["name"]

'Kudensee'

In [24]:
send1 = Leipig["locations.id"]
send2 = Leipig["locations.productionLocationAndRoutingCodes.productionLocation.id"]
rec1 = Berlin["locations.id"]
rec2 = Berlin["locations.productionLocationAndRoutingCodes.productionLocation.id"]
rc = Berlin["locations.productionLocationAndRoutingCodes.routingCodes"]

# r = get_connection_synchronus(send1,send2,rec1,rec2,rc)
# r.json()

In [25]:
import random
random.random()

0.260982443422596

es schein so, als ob man echt nicht mehr als 1 anfrage pro sekunde durchbekommen kann, da sonst errors/ratelimits auftauchen

manchmal gibt es keine json zurück

möglicher usecase wäre zB nur alle Güterzüge von Leipzig-Wahren abzufragen

In [60]:
import asyncio
import httpx

results = []
# Funktion, um Anfragen zu senden und Antworten in eine Liste zu sammeln
async def get_connection(send1, send2, rec1, rec2, code, results):
    
    base = "https://planner.dbcargo.com/nwd/timetableQuery"
    payload = {
        "dispatchLocationId" : str(send1),
        "departureLocationId": str(send2),
        "receivingLocationId" : str(rec1),
        "arrivalLocationId" : str(rec2),
        "routingCode" : str(code),
        "earliestTimeOfDay" : 0,
        "latestTimeOfDay" : 86399,
        "mappingType": "Partner",
        "scenarioHash": "2d2996d25ff0d332744c3d1310942e8dc6256396d3432aff986b6664359047ed",
        "searchMode": "Departure",
        "weekdays": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    }

    # Setze einen Timeout-Wert von 30 Sekunden (oder beliebig länger)
    timeout = httpx.Timeout(30.0)

    async with httpx.AsyncClient(timeout=timeout) as client:
        response = {}
        response = await client.post(url=base, json=payload)

        i = 0
        while "scenarioInformation" not  in response.json().keys():
            response = await client.post(url=base, json=payload)
            # falloff = random.random() * (1.5 ** i)
            # print(falloff)
            # time.sleep(falloff)
            i += 1
            if i > 3:
                break

        results.append(response.json())


# Funktion zum Verarbeiten der Ergebnisse
async def main():
    send1 = Leipig["locations.id"]
    send2 = Leipig["locations.productionLocationAndRoutingCodes.productionLocation.id"]
    rec1 = Berlin["locations.id"]
    rec2 = Berlin["locations.productionLocationAndRoutingCodes.productionLocation.id"]
    code = Berlin["locations.productionLocationAndRoutingCodes.routingCodes"]
    # Eine Liste für alle Antworten

    # Liste der Aufgaben, die sofort gestartet werden
    # tasks = [
    #     asyncio.create_task(get_connection(send1, send2, rec1, rec2, code, results)) 
    #     for _ in range(100)  # Beispiel für 10 Anfragen
    # ]
    tasks = []
    for i in range(10):
        send1 = Leipig["locations.id"]
        send2 = Leipig["locations.productionLocationAndRoutingCodes.productionLocation.id"]
        rec1 = companys.iloc[i+1]["locations.id"]
        rec2 = companys.iloc[i+1]["locations.productionLocationAndRoutingCodes.productionLocation.id"]
        code = companys.iloc[i+1]["locations.productionLocationAndRoutingCodes.routingCodes"]
        tasks.append(asyncio.create_task(get_connection(send1, send2, rec1, rec2, code, results)))
    # Warte, dass alle Aufgaben abgeschlossen sind
    await asyncio.gather(*tasks)

    # Ausgabe aller gesammelten Antworten
    print("Alle gesammelten Antworten:")
    for response in results:
        print(response)

# In Jupyter Notebook kannst du die Hauptfunktion direkt aufrufen:
await main()



Alle gesammelten Antworten:
{'error': True}
{'error': True}
{'error': True}
{'scenarioInformation': {'scenarioHash': '2d2996d25ff0d332744c3d1310942e8dc6256396d3432aff986b6664359047ed', 'validFrom': '2025-03-03T00:00:00+01:00', 'validUntil': '2025-04-08T00:00:00+02:00', 'referenceWeekBegin': '2025-03-03T00:00:00+01:00', 'referenceWeekEnd': '2025-03-09T00:00:00+01:00', 'timetableTimestamp': '2025-02-28T13:11:23.170156+01:00'}, 'connectionRequestParameterSet': {'dispatchLocationId': 22930252, 'receivingLocationId': 22929924, 'departureLocationId': 22922003, 'arrivalLocationId': 22920476, 'routingCode': '020-21', 'searchMode': 'Departure', 'mappingType': 'Partner', 'weekdays': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 'earliestTimeOfDay': 0, 'latestTimeOfDay': 86399}, 'connections': []}
{'scenarioInformation': {'scenarioHash': '2d2996d25ff0d332744c3d1310942e8dc6256396d3432aff986b6664359047ed', 'validFrom': '2025-03-03T00:00:00+01:00', 'validUntil': '20

In [61]:
counter = 0
for r in results:
    if "scenarioInformation" in r.keys():
        counter += 1
print(counter)

6


In [29]:
import plotly.express as px

In [30]:
new_df

,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,locations.productionLocationAndRoutingCodes.productionLocation.locationNumber,locations.productionLocationAndRoutingCodes.productionLocation.name,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,010819,HAMBURG-WALTERSHOF,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
1,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010066,None,AKUD,22920361.0,80,010066,KUDENSEE,...,22920413.0,80,010066,KUDENSEE,53.922657,9.219641,KUDENSEE,1.0,NaN,NaN
2,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,011759,MASCHEN RBF,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,011759,MASCHEN RBF,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,010660,HAMBURG-BILLWERDER,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2210,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2211,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2212,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [31]:
companys

,id,name,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
1,010066,Kudensee,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010066,None,AKUD,22920361,80,...,22920413,80,010066,KUDENSEE,53.922657,9.219641,KUDENSEE,1.0,NaN,NaN
9,011205,Hamburg-Barmbek,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011205,None,ABAG,22920426,80,...,22920427,80,011205,HAMBURG-BARMBEK,53.597759,10.036916,HAMBURG-BARMBEK,1.0,NaN,NaN
12,011460,Dannenberg Ost,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011460,None,ADB,22917069,80,...,22929266,80,011460,DANNENBERG OST,53.107161,11.120125,DANNENBERG OST,1.0,NaN,NaN
13,011742,Lüneburg,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011742,None,ALBG,22920421,80,...,22929263,80,011742,LUENEBURG,53.250934,10.420770,LUENEBURG,2.0,NaN,NaN
16,011833,Meckelfeld,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00011833,None,AMD,22920462,80,...,22920463,80,011833,MECKELFELD,53.421757,10.029219,MECKELFELD,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,893651,Dingolfing PA Scholz,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00893651,None,MADS,22915095,80,...,22915096,80,893651,DINGOLFING PA SCHOLZ,48.642839,12.491984,DINGOLFING PA SCHOLZ,1.0,NaN,NaN
2201,894725,Bülstringen Florastrasse,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00894725,None,LFLO,22915092,80,...,22915093,80,894725,BUELSTRINGEN FLORAST,52.311128,11.333092,BUELSTRINGEN FLORAST,1.0,NaN,NaN
2202,897926,Dabel,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00897926,None,WDL,22915088,80,...,22915089,80,897926,DABEL,53.671404,11.893348,DABEL,2.0,NaN,NaN
2203,899385,Krefeld Industriebahn,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00899385,None,KKRK,22919136,80,...,22930374,80,899385,KREFELD INDUSTRIEBHN,51.342592,6.591727,KREFELD INDUSTRIEBHN,1.0,NaN,NaN


In [32]:
fig = px.scatter(companys, y="locations.productionLocationAndRoutingCodes.productionLocation.latitude",
                  x="locations.productionLocationAndRoutingCodes.productionLocation.longitude", 
                  title="Interaktiver 2D-Scatterplot",
                  hover_data=["name"])
fig.update_layout(
    xaxis=dict(scaleanchor="y"),  # X- und Y-Skala verknüpfen
    yaxis=dict(scaleanchor="x"),
    width=800,  # Quadrat erzwingen
    height=800
)
fig.update_traces(
    hovertemplate='%{customdata}<extra></extra>'  # Zeigt nur den Namen an
)

# Custom-Daten für den Namen hinzufügen
fig.update_traces(customdata=companys["name"])

fig.show()

In [33]:
df_duplicates = combined[combined.duplicated(subset=["locations.productionLocationAndRoutingCodes.productionLocation.latitude"], keep=False)]
df_duplicates

,id,name,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,010033,Hamburg-Who Ct Burchardkai Ctb,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
2,010116,Maschen MVZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,010124,Maschen Rbf Werk Y-Schad,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,010660,Hamburg-Billwerder Ubf,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
5,010777,Cuxhaven Fischereihafen Cuxport-Terminal,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010777,None,ACV,22920465.0,80,...,22929272.0,80,010777,CUXHAVEN FISCH CUX T,53.849509,8.726545,CUXHAVEN FISCH CUX T,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,999953,MALUS,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2210,999961,KULANZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2211,999979,LOGISTIK,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2212,999987,CZ BLN SERVICE ABEGL,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [34]:
df_duplicates2 =  df_duplicates[df_duplicates.duplicated(subset=["locations.productionLocationAndRoutingCodes.productionLocation.latitude"], keep=False)]

In [35]:
df_duplicates

,id,name,scenarioHash,locations.companyLocationNumberOwner,locations.type,locations.tafTsiCode,locations.productionLocationAndRoutingCodes.productionLocation.freightCarLocationNumber,locations.productionLocationAndRoutingCodes.productionLocation.code,locations.productionLocationAndRoutingCodes.productionLocation.id,locations.productionLocationAndRoutingCodes.productionLocation.countryNumber,...,locations.id,locations.countryNumber,locations.locationNumber,locations.name,locations.latitude,locations.longitude,locations.objectKeyAlpha,locations.objectKeySequence,locations.productionLocationAndRoutingCodes,locations
0,010033,Hamburg-Who Ct Burchardkai Ctb,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010033,None,AWHOD,22920434.0,80,...,22929273.0,80,010033,HAMBURG-WALTERS BUC,53.531398,9.917442,HAMBURG-WHO CTB,1.0,NaN,NaN
2,010116,Maschen MVZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010116,None,AM,22920469.0,80,...,22929274.0,80,010116,MASCHEN MVZ,53.418000,10.037300,MASCHEN MVZ,1.0,NaN,NaN
3,010124,Maschen Rbf Werk Y-Schad,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010124,None,AM,22920469.0,80,...,22929271.0,80,010124,MASCHEN WERK Y-SCHAD,53.399715,10.055368,MASCHEN RBF N/S KV,1.0,NaN,NaN
4,010660,Hamburg-Billwerder Ubf,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010660,None,AHBI,22920359.0,80,...,22920360.0,80,010660,HAMBURG-BILLW UBF,53.516175,10.093603,HAMBURG-BILLW UBF,1.0,NaN,NaN
5,010777,Cuxhaven Fischereihafen Cuxport-Terminal,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,86353173,None,00010777,None,ACV,22920465.0,80,...,22929272.0,80,010777,CUXHAVEN FISCH CUX T,53.849509,8.726545,CUXHAVEN FISCH CUX T,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,999953,MALUS,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2210,999961,KULANZ,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2211,999979,LOGISTIK,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2212,999987,CZ BLN SERVICE ABEGL,2d2996d25ff0d332744c3d1310942e8dc6256396d3432a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [36]:
df_duplicates2.shape

(1095, 27)